# 卷积神经网络(CNN)
通过卷积神经网络对MNIST数据集处理

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
print("number of train data is %d"%(mnist.train.num_examples))
print("number of test data is %d"%(mnist.test.num_examples))
trainimg=mnist.train.images
trainlabel=mnist.train.labels
testimg=mnist.test.images
testlabel=mnist.test.labels
print("MNIST ready")

W1113 15:17:33.920734 10700 deprecation.py:323] From <ipython-input-3-e1dd9a9b7511>:4: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1113 15:17:33.929707 10700 deprecation.py:323] From D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1113 15:17:33.945666 10700 deprecation.py:323] From D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
number of train data is 55000
number of test data is 10000
MNIST ready


### 构建卷积神经网络模型

卷积神经网络模型的构建主要分为四步，构建输入层、隐层、输出层以及定义损失函数。

对于隐层，我们构建了卷积层1-池化层1-卷积层2-池化层2-全连接神经网络层-Dropout层。

1. 输出层：我们需要明确输入数据的维度即可，对于MNIST数据集来说，输入的是一张维度为28*28=784的灰度图，灰度图的通道是1

2. 卷积层1：和全神经网络中输出值的处理类似，都需要依据权重和偏置项进行激活函数处理，可以表示为：
$ y=f(conv(权重\times x)+偏置项) $。
首先运行卷积函数，然后计算偏置项，最后进行激活函数的处理。
其中input为输入层的输入数据，维度为$[batch,in_height,in_width,in_channels]$，
需要获取输入图像的数量、高度、宽度以及通道数。对于数量不考虑，标识-1；图片是28像素*28像素的灰度图，灰度图通道为1；如果是RGB彩图，通道为3，输入数据为
$[-1,28,28,1]$

3. 全连接神经网络在经过池化层2处理后的所有神经元节点作为全神经网络的输入神经元，所以关键就是获得池化层2处理后的神经元节点的个数。
在池化层2处理后，总神经元节点数的计算公式为：单个特征映射$\times$特征映射个数。其中特征映射个数为64，单个特征映射由原始图像的28$\times$28，经过一次池化后为14$\times$14，
经过第二次池化后为7$\times$7，总神经元节点数为7$\times$7$\times$64个。

4. Dropot层，为了防止过拟合使用的，目的是让神经网络中的神经元随机出现“休眠”，让这些“休眠”的神经元不参与本次模型的运算。降低过拟合。


In [4]:
n_input=784
n_output=10
# x=tf.placeholder(tf.float32,[None,n_input])
# 卷积层输入数据
# x_image=tf.reshape(x,[-1,28,28,1])
# 卷积层定义
# W_conv1=[3,3,1,32]
# 平移步长
# strides=[1,1,1,1]
# tf.nn.conv2d(x,W_conv1,strides,padding='SAME')
# 权重值初始化函数
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
# 偏置项初始化函数
def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)
# 卷积计算函数
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
# 池化窗口定义
ksize=[1,2,2,1]
# 步长定义，在batch和channels上不进行池化，维度设置为1
strides=[1,2,2,1]
# 池化层函数定义(池化层使用卷积层的输出值)
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize,strides,padding='SAME')
# 定义卷积神经网络函数
def CNN_mnist(x):
    # 定义图像数据
    with tf.name_scope('reshape'):
        x_image=tf.reshape(x,[-1,28,28,1])
        
    # 定义卷积层1和池化层1
    # 实现卷积层1
    with tf.name_scope('conv1'):
        # 使用卷积核作为权重shape
        W_conv1=weight_variable([3,3,1,32])
        b_conv1=bias_variable([32])
        # 对卷积核函数进行激活处理
        h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
    # 池化层1中的具体实现(池化层使用卷积层的输出值)
    with tf.name_scope('pool1'):
        h_pool1=max_pool_2x2(h_conv1)
        
    
    # 定义卷积层2和池化层2
    # 实现卷积层2(输入使用上一层池化层的输出值)，32表示上一层核函数个数即32个特征映射，
    # 即图像通道个数为32.而卷积核的个数必须在上一层的基础上有所增加，使用64
    with tf.name_scope('conv2'):
        W_conv2=weight_variable([3,3,32,64])
        b_conv2=bias_variable([64])
        #  对卷积核函数进行激活处理
        h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
    # 池化层2中的具体实现
    with tf.name_scope('pool2'):
        h_pool2=max_pool_2x2(h_conv2)
        
    
    # 构建全连接神经网络层的实现
    # 定义fc1，将两次池化后的神经元转换为1D向量
    with tf.name_scope('fc1'):
        W_fc1=weight_variable([7*7*64,1024])
        b_fc1=bias_variable([1024])
        h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
        h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
        
    
    # Dropout层
    # 让神经元随机休眠，防止过拟合
    with tf.name_scope('dropout'):
        keep_prob=tf.placeholder(tf.float32)
        h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)
    
    
    
    # 输出层
    with tf.name_scope('fc2'):
        W_fc2=weight_variable([1024,10])
        b_fc2=bias_variable([10])
        # 输出不使用激活函数
        y_conv=tf.matmul(h_fc1_drop,W_fc2)+b_fc2
        print("CNN READY")
    return y_conv,keep_prob


# 损失函数，使用最长用的softmax交叉熵损失函数
x=tf.placeholder(tf.float32,[None,n_input])
y_=tf.placeholder(tf.float32,[None,n_output])
# 进行卷积神经网络训练
y_conv,keep_prob=CNN_mnist(x)
# 定义损失
with tf.name_scope('loss'):
    cross_entropy=tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_conv)
    cross_entropy=tf.reduce_mean(cross_entropy)
# 优化器
with tf.name_scope('adam_optimizer'):
    # 函数是Adam优化算法：是一个寻找全局最优点的优化算法，引入了二次方梯度校正。
    # 自适应优化器，包括tf.train.AdagradOptimizer和tf.train.AdamOptimizer，这些可以作为随时可用的替代品
    train_step=tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
    # 旨在对所有步骤中的所有变量使用恒定的学习率。 
    # train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    


W1113 15:17:41.433973 10700 deprecation.py:506] From <ipython-input-4-9e4f393dba0d>:70: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1113 15:17:41.554649 10700 deprecation.py:323] From <ipython-input-4-9e4f393dba0d>:90: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



CNN READY


进行数据训练

In [7]:
# 定义评测准确率的操作
with tf.name_scope('accuracy'):
    correct_prediction=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
    correct_prediction=tf.cast(correct_prediction,tf.float32)
    accuracy=tf.reduce_mean(correct_prediction)
# 初始化所有参数
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    # 所有样本迭代1000次
    training_epochs=1001
    # 每次选择100个样本
    batch_size=100
    display_step=100
    for i in range(training_epochs):
        avg_cost=0
        total_batch=int(mnist.train.num_examples/batch_size)
        batch=mnist.train.next_batch(batch_size)
        # 每个元素被保留下来的概率为0.7
        sess.run(train_step,feed_dict={x:batch[0],y_:batch[1],keep_prob:0.7})
        
        # 评估模型,每100次评估一次
        if i%display_step==0:
            # t.eval()等效于sess.run(t).
            train_accuracy=sess.run(accuracy,feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
            print('step %d,training accuracy %g'%(i,train_accuracy))
            test_accuracy=accuracy.eval(feed_dict={x:mnist.test.images,
                                                   y_:mnist.test.labels,
                                                   keep_prob:1.0})
            print('test accuracy %g'%(test_accuracy))

step 0,training accuracy 0.13
test accuracy 0.1009
step 100,training accuracy 0.89
test accuracy 0.8939
step 200,training accuracy 0.92
test accuracy 0.9215
step 300,training accuracy 0.94
test accuracy 0.9363
step 400,training accuracy 0.95
test accuracy 0.9435
step 500,training accuracy 0.94
test accuracy 0.9507
step 600,training accuracy 0.99
test accuracy 0.9516
step 700,training accuracy 0.98
test accuracy 0.9484
step 800,training accuracy 0.92
test accuracy 0.9461
step 900,training accuracy 0.97
test accuracy 0.9529
step 1000,training accuracy 0.93
test accuracy 0.9518
